# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation, profile_information, project_information
from madminer.plotting import plot_fisher_information_contours_2d


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.DEBUG
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma_sys/'
result_dir = base_dir + 'data/results/wgamma_sys/'

## Settings

In [5]:
batch_size=10000

## Main functions

In [6]:
fisher_all = FisherInformation(sample_dir + 'samples.h5')
fisher_tight = FisherInformation(sample_dir + 'samples_tight.h5')

11:16 madminer.fisherinfor INFO    Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples.h5
11:16 madminer.fisherinfor INFO    Found 2 parameters
11:16 madminer.fisherinfor DEBUG      cWWW (LHA: dim6 1, maximal power in squared ME: (2,), range: (-0.02, 0.02))
11:16 madminer.fisherinfor DEBUG      cWWWtilde (LHA: dim6 2, maximal power in squared ME: (2,), range: (-0.02, 0.02))
11:16 madminer.fisherinfor INFO    Found 55 nuisance parameters
11:16 madminer.fisherinfor DEBUG      mur (('35', '6'))
11:16 madminer.fisherinfor DEBUG      muf (('25', '16'))
11:16 madminer.fisherinfor DEBUG      pdf_0 (('45', None))
11:16 madminer.fisherinfor DEBUG      pdf_1 (('46', None))
11:16 madminer.fisherinfor DEBUG      pdf_2 (('47', None))
11:16 madminer.fisherinfor DEBUG      pdf_3 (('48', None))
11:16 madminer.fisherinfor DEBUG      pdf_4 (('49', None))
11:16 madminer.fisherinfor DEBUG      pdf_5 (('50', None))
11:16 madminer.fisherinfor DEBUG   

11:16 madminer.fisherinfor DEBUG      66: nuisance parameter
11:16 madminer.fisherinfor DEBUG      67: nuisance parameter
11:16 madminer.fisherinfor DEBUG      68: nuisance parameter
11:16 madminer.fisherinfor DEBUG      69: nuisance parameter
11:16 madminer.fisherinfor DEBUG      7: nuisance parameter
11:16 madminer.fisherinfor DEBUG      70: nuisance parameter
11:16 madminer.fisherinfor DEBUG      71: nuisance parameter
11:16 madminer.fisherinfor DEBUG      72: nuisance parameter
11:16 madminer.fisherinfor DEBUG      73: nuisance parameter
11:16 madminer.fisherinfor DEBUG      74: nuisance parameter
11:16 madminer.fisherinfor DEBUG      75: nuisance parameter
11:16 madminer.fisherinfor DEBUG      76: nuisance parameter
11:16 madminer.fisherinfor DEBUG      77: nuisance parameter
11:16 madminer.fisherinfor DEBUG      78: nuisance parameter
11:16 madminer.fisherinfor DEBUG      79: nuisance parameter
11:16 madminer.fisherinfor DEBUG      8: nuisance parameter
11:16 madminer.fisherinfor

11:16 madminer.fisherinfor DEBUG      26: nuisance parameter
11:16 madminer.fisherinfor DEBUG      27: nuisance parameter
11:16 madminer.fisherinfor DEBUG      28: nuisance parameter
11:16 madminer.fisherinfor DEBUG      29: nuisance parameter
11:16 madminer.fisherinfor DEBUG      3: nuisance parameter
11:16 madminer.fisherinfor DEBUG      30: nuisance parameter
11:16 madminer.fisherinfor DEBUG      31: nuisance parameter
11:16 madminer.fisherinfor DEBUG      32: nuisance parameter
11:16 madminer.fisherinfor DEBUG      33: nuisance parameter
11:16 madminer.fisherinfor DEBUG      34: nuisance parameter
11:16 madminer.fisherinfor DEBUG      35: nuisance parameter
11:16 madminer.fisherinfor DEBUG      36: nuisance parameter
11:16 madminer.fisherinfor DEBUG      37: nuisance parameter
11:16 madminer.fisherinfor DEBUG      38: nuisance parameter
11:16 madminer.fisherinfor DEBUG      39: nuisance parameter
11:16 madminer.fisherinfor DEBUG      4: nuisance parameter
11:16 madminer.fisherinfor

In [7]:
def calculate_info(filename, calculation_mode, tight_cuts=False, **kwargs):
    if tight_cuts:
        fisher_ = fisher_tight
        ending_ = '_tight.npy'
    else:
        fisher_ =  fisher_all
        ending_ = '.npy'
    
    if calculation_mode == "truth":
        fisher_info, cov = fisher_.calculate_fisher_information_full_truth(
            theta=[0.,0.],
            luminosity = 3000000.,
            **kwargs
        )
    elif calculation_mode == "rate":
        fisher_info, cov = fisher_.calculate_fisher_information_rate(
            theta=[0.,0.],
            luminosity = 3000000.,
            **kwargs
        )
    elif calculation_mode == "histo":
        fisher_info, cov = fisher_.calculate_fisher_information_hist1d(
            theta=[0.,0.],
            luminosity = 3000000.,
            **kwargs
        )
    elif calculation_mode == "histo2d":
        fisher_info, cov = fisher_.calculate_fisher_information_hist2d(
            theta=[0.,0.],
            luminosity = 3000000.,
            **kwargs
        )
    elif calculation_mode == "ml":
        fisher_info, cov = fisher_.calculate_fisher_information_full_detector(
            theta=[0.,0.],
            luminosity = 3000000.,
            **kwargs
        )

    np.save(result_dir + 'information_{}{}'.format(filename, ending_), fisher_info)
    np.save(result_dir + 'covariance_{}{}'.format(filename, ending_), cov)

    fisher_info_nuisance = fisher_.calculate_fisher_information_nuisance_constraints()
    fisher_info, cov = profile_information(fisher_info + fisher_info_nuisance, [0,1], covariance=cov)
    
    np.save(result_dir + 'profiled_information_{}{}'.format(filename, ending_), fisher_info)
    np.save(result_dir + 'profiled_covariance_{}{}'.format(filename, ending_), cov)


## Truth-level info

In [8]:
calculate_info('parton', 'truth', False)
calculate_info('parton', 'truth', True)

/Users/johannbrehmer/work/projects/madminer/madminer/madminer/morphing.py:695: RuntimeWarning: divide by zero encountered in log
  a.append(np.log(benchmark_weights[:, i_pos] / benchmark_weights[:, self.i_benchmark_ref]))
/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:127: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  size=error_propagation_n_ensemble,


## Rate-only info

In [9]:
calculate_info('rate', 'rate', False)
calculate_info('rate', 'rate', True)

## Full Fisher info (ML)

In [10]:
calculate_info('full', 'ml', False, model_file=model_dir + "sally_ensemble_all", mode="score")
calculate_info('full', 'ml', True, model_file=model_dir + "sally_ensemble_all_tight", mode="score", batch_size=batch_size)


21:36 madminer.ml          INFO    Found ensemble with 10 estimators and expectations None
21:38 madminer.fisherinfor INFO    Evaluating rate Fisher information
21:40 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 1 / 9
21:40 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 2 / 9
21:40 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 3 / 9
21:40 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 4 / 9
21:41 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 5 / 9
21:41 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 6 / 9
21:41 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 7 / 9
21:41 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 8 / 9
21:42 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 9 / 9
21:42 madminer.ml          IN

In [11]:
calculate_info('full_infomode', 'ml', False, model_file=model_dir + "sally_ensemble_all", mode="information")
calculate_info('full_infomode', 'ml', True, model_file=model_dir + "sally_ensemble_all_tight", mode="information", batch_size=batch_size)


21:46 madminer.ml          INFO    Found ensemble with 10 estimators and expectations None
21:47 madminer.fisherinfor INFO    Evaluating rate Fisher information
21:50 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 1 / 9
21:50 madminer.ml          INFO    Expected per-event score (should be close to zero): [ 3.11584687e+00  4.89694881e+00  5.21829861e-07 -1.59094736e-01
  1.21785695e-06  3.57532292e-03 -3.06561077e-03  1.16358362e-02
 -1.20623829e-02 -3.53403762e-02  3.00516430e-02  2.87841889e-03
 -1.58767973e-03 -9.86734312e-03  1.02919545e-02  4.71285358e-03
 -4.32289438e-03  2.15152465e-02 -2.87677571e-02  2.11884663e-03
 -2.87354062e-03 -4.11842531e-03 -1.25841936e-03 -4.35077436e-02
  1.86304972e-02  2.56039482e-03 -6.55575050e-03 -4.29936033e-03
  1.43662561e-02  3.17927846e-03 -8.14211648e-03  1.13001186e-03
  8.09564139e-04  1.13209020e-02 -1.07691092e-02 -2.34980006e-02
  1.93700083e-02  9.12109576e-03  5.35145635e-04  4.27487167e-03
 -6.89655263

21:50 madminer.ml          INFO    Expected per-event score (should be close to zero): [ 1.10409174e+01  3.36916256e+00  3.40393342e-07 -1.41887203e-01
  9.02141949e-07  6.33289991e-03 -5.26627619e-03  9.99069773e-03
 -1.00958841e-02 -2.68870778e-02  2.25668829e-02  1.22997360e-02
 -1.15488647e-02 -4.87349741e-03  5.94152417e-03  2.66975863e-03
 -2.25952314e-03  1.62800569e-02 -2.23771408e-02 -1.88203738e-03
  1.41425966e-03  1.57044420e-03 -2.29273061e-03 -2.82457732e-02
  1.04856985e-02 -8.12859973e-04 -1.92089018e-03 -4.36564907e-03
  1.32156080e-02  3.64615023e-03 -9.69960727e-03 -1.05946769e-04
  2.39836005e-03  1.13921091e-02 -8.75078700e-03 -1.54761104e-02
  1.37457615e-02  9.44894925e-03  6.77369128e-04  2.83376011e-03
 -2.36575818e-03  1.06658489e-02 -1.32431705e-02 -7.15673203e-03
  5.55946818e-03  3.66941863e-03 -5.85084269e-03  1.61916600e-04
 -2.20562005e-03  5.07744495e-04 -9.78036039e-03 -2.98308814e-03
  1.72830746e-02  2.25953008e-05  5.11305220e-03 -4.34925826e-03
  3

21:50 madminer.ml          INFO    Expected per-event score (should be close to zero): [-1.5462914e+00  1.1466296e+01  1.0361034e-05 -1.4081489e-01
 -1.1747481e-07  4.6596033e-03 -4.5196754e-03  9.1155637e-03
 -9.2468914e-03 -2.7072068e-02  2.2692012e-02  5.8065127e-03
 -5.6906459e-03 -7.0011485e-03  8.5210158e-03  5.4868837e-03
 -4.2760130e-03  1.3798415e-02 -1.9029196e-02  4.4317727e-04
 -8.5714867e-04 -7.6460524e-04 -3.4812789e-03 -2.8242990e-02
  9.5968517e-03  7.0105051e-04 -3.2022025e-03 -2.8645014e-03
  1.2666414e-02  2.2065507e-03 -5.8411970e-03 -1.5533274e-03
  2.1005447e-03  1.1725426e-02 -1.0139700e-02 -1.2576236e-02
  9.5178885e-03  5.4426361e-03  2.2553573e-03  5.1676673e-03
 -4.2329328e-03  7.3905727e-03 -1.1770328e-02 -6.6362075e-03
  4.7229771e-03  5.1598148e-03 -7.0000486e-03 -3.1943882e-03
 -2.9934171e-04  1.6739423e-03 -1.1227052e-02 -1.5245882e-03
  1.5416838e-02  6.5254461e-04  3.3640256e-03 -5.4238453e-03
  1.4736437e-03]
21:50 madminer.ml          INFO    Expecte

21:51 madminer.ml          INFO    Expected per-event score (should be close to zero): [-1.0163191e+01 -1.5976799e+00  1.4016460e-05 -1.3639539e-01
  3.9771556e-05  4.0478716e-03 -3.7376049e-03  1.0903130e-02
 -1.0818889e-02 -2.4507564e-02  2.3049165e-02  9.3743997e-03
 -8.4159337e-03 -6.2742382e-03  7.3519545e-03 -1.2523130e-03
  8.0330219e-06  1.5575267e-02 -2.0774988e-02 -2.0305673e-03
  1.5192318e-03  3.9744471e-05 -3.4971447e-03 -2.7666261e-02
  9.8907845e-03  8.0815665e-03 -9.8615643e-03 -3.0528726e-03
  1.3985151e-02  5.4028351e-03 -1.0519780e-02 -5.9937090e-03
  5.7726097e-03  1.1882284e-02 -1.0407574e-02 -1.5211083e-02
  1.7114824e-02  7.7339136e-03  1.8747930e-03  5.0084968e-03
 -2.9334461e-03  1.4858848e-02 -1.6708614e-02 -2.5238690e-03
  2.9545422e-03 -4.5470535e-04 -3.0335053e-03 -5.2397716e-04
 -1.7392564e-03  5.5595296e-03 -1.4700025e-02 -2.7702500e-03
  1.8740572e-02  4.0707211e-03  1.7999832e-03 -2.8301515e-03
  4.0868312e-04]
21:51 madminer.ml          INFO    Expecte

21:51 madminer.ml          INFO    Expected per-event score (should be close to zero): [-1.82219276e+01  1.19054103e+00 -1.89372757e-03 -1.26513466e-01
 -4.63715143e-04  3.28958943e-03 -4.58866358e-03  8.91445298e-03
 -9.29624587e-03 -2.45449394e-02  2.11899970e-02  7.62981828e-03
 -7.51854200e-03 -7.01781269e-03  8.48576333e-03  4.14038403e-03
 -1.86044292e-03  1.35340113e-02 -1.65537447e-02 -2.22167931e-03
  2.13862024e-03 -2.70376389e-04 -2.61756568e-03 -2.69901175e-02
  1.18806278e-02  7.77971349e-04 -3.88541468e-03  4.42743214e-04
  1.03830602e-02  7.14973221e-03 -9.92953405e-03 -2.29226029e-03
  1.14688475e-03  1.35554103e-02 -1.01747038e-02 -1.44725265e-02
  1.21564474e-02  4.91058268e-03  4.04317718e-04  3.19752097e-03
 -3.40266386e-03  1.19491341e-02 -1.44246509e-02 -5.73319755e-03
  4.44533397e-03  5.74296061e-03 -8.73634964e-03 -5.19707857e-04
  1.55063797e-04  1.38127652e-03 -7.20936572e-03 -2.35260837e-03
  1.58807542e-02  8.98396538e-04  6.10424858e-03 -5.33525413e-03
  2

...

21:58 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 54 / 57
21:58 madminer.ml          INFO    Expected per-event score (should be close to zero): [-1.29664927e+01  3.65983868e+00  1.00158024e-04 -2.69125830e-02
  1.35789451e-05  1.16640294e-03 -1.39626628e-03  2.97551975e-03
 -2.88519193e-03 -4.43426659e-03  3.50756501e-03 -1.52524700e-03
  1.74644752e-03 -5.23785874e-03  4.87313187e-03 -3.36548983e-04
 -2.91391439e-03  1.13994945e-02 -1.77202169e-02  2.82668672e-03
 -3.51402233e-03  6.93511358e-03 -4.84743435e-03 -1.02497283e-02
  4.04089224e-03  3.01977787e-02 -2.86316127e-02 -1.36408918e-02
  1.78906191e-02  6.99772639e-03 -1.24235665e-02 -3.21460073e-03
  2.75720027e-03 -2.99276318e-03  1.73058012e-03 -9.51554906e-03
  7.38317566e-03  2.11777887e-03  3.66867823e-03 -8.48077144e-03
  3.78409307e-03 -1.15815215e-02  1.10152438e-02 -6.93473732e-04
  1.73438317e-03 -8.07674194e-04  1.68885326e-03 -8.16757558e-04
  7.48779508e-04 -6.28016796e-03  7.47254

21:58 madminer.ml          INFO    Expected per-event score (should be close to zero): [-1.8352940e+01  1.5133269e+00 -1.4867232e-05 -2.6975913e-02
  1.4371359e-04  1.0229674e-03 -1.2122596e-03  2.7845700e-03
 -3.0968164e-03 -6.1560804e-03  4.7263573e-03 -3.1452272e-03
  3.1819781e-03 -8.0229584e-03  8.7295994e-03  2.1353248e-03
 -3.3943909e-03  1.1071219e-02 -1.6013967e-02 -1.8776766e-03
  1.3694190e-04  5.3040991e-03 -3.0338473e-03 -8.7580010e-03
  3.9162328e-03  3.0773669e-02 -3.0973330e-02 -9.3144439e-03
  1.3209941e-02  6.5785926e-03 -1.6391011e-02 -7.0248544e-03
  5.3768591e-03 -5.5824243e-03  2.9029748e-03 -1.2032972e-02
  8.6069722e-03  2.7251940e-03  4.1505247e-03 -7.6598418e-03
  4.7383010e-03 -1.3407176e-02  9.4359843e-03  5.0989114e-05
  4.3009335e-04 -1.3952198e-03  1.3914654e-03 -4.4503766e-03
  1.2955583e-03 -5.8308104e-03  7.5149438e-03  2.7907905e-04
  1.3544219e-02 -7.9516591e-05  2.8791274e-03 -5.1049277e-04
 -1.1336305e-03]
21:58 madminer.ml          INFO    Expecte

21:58 madminer.ml          INFO    Expected per-event score (should be close to zero): [-1.0700064e+01 -4.7646794e+00  2.6610540e-04 -3.3321880e-02
  1.6678215e-04  1.4749514e-03 -1.0052518e-03  3.3091542e-03
 -3.9297920e-03 -7.3430841e-03  5.2511641e-03 -4.1870736e-03
  4.6014753e-03 -1.3260311e-02  1.4749124e-02  6.2063541e-03
 -7.7044363e-03  1.2703798e-02 -1.8297844e-02 -9.0458561e-03
  5.1769083e-03  4.4363108e-04 -3.1484191e-03 -1.2083919e-02
  4.5337216e-03  4.0600669e-02 -3.8282357e-02 -1.0770841e-02
  1.6673759e-02  1.2904287e-02 -2.6209904e-02 -9.2361392e-03
  7.4864584e-03 -9.3822517e-03  2.9014978e-03 -1.8496696e-02
  1.2396567e-02  4.0864567e-03  3.9407485e-03 -1.4100479e-02
  6.3095312e-03 -2.1219900e-02  1.5346120e-02 -6.3691806e-04
 -1.4251175e-03 -9.5933973e-04  4.0471063e-05 -8.6845085e-03
 -4.5556717e-05 -1.0662663e-02  1.3138721e-02  4.2963790e-04
  1.6702268e-02  4.5072993e-05  3.7047255e-04  1.5256945e-03
  9.2750537e-04]
21:58 madminer.ml          INFO    Expecte

21:58 madminer.ml          INFO    Expected per-event score (should be close to zero): [-1.6284351e+01  3.3713222e+00 -3.7967216e-04 -3.1505272e-02
 -4.0605039e-04  8.1592123e-04 -1.3124961e-03  2.8172396e-03
 -3.0191632e-03 -5.8312388e-03  4.8910589e-03 -3.1462833e-03
  4.0666903e-03 -1.1242433e-02  1.2401295e-02  3.5504918e-03
 -5.6426018e-03  1.2856061e-02 -1.7794492e-02 -6.1150263e-03
  3.4250449e-03 -9.1614848e-04 -2.5156394e-03 -1.7699929e-02
  6.2427111e-03  3.8393181e-02 -3.6305234e-02 -1.2075279e-02
  1.7124409e-02  1.0248287e-02 -2.5639493e-02 -8.5955868e-03
  6.0063871e-03 -7.1068299e-03  9.2134962e-04 -1.4141300e-02
  1.0559536e-02  3.2559037e-03  2.8027256e-03 -1.0573963e-02
  4.9073072e-03 -1.4471188e-02  1.2474906e-02  4.1962089e-04
 -1.0996119e-03 -1.6399598e-03  2.3381400e-03 -4.6945489e-03
 -2.7112986e-04 -7.4448534e-03  1.1503411e-02  1.6201251e-03
  1.4260945e-02 -5.2647601e-04  1.0580185e-03 -5.8349292e-04
  7.0797032e-05]
21:58 madminer.ml          INFO    Expecte

21:58 madminer.ml          INFO    Expected per-event score (should be close to zero): [-1.28466864e+01  2.61390984e-01  1.60806288e-04 -3.48290801e-02
 -3.08398245e-04  8.70739284e-04 -8.84542125e-04  4.36275732e-03
 -4.13913466e-03 -7.17210444e-03  5.70723647e-03 -3.47466860e-03
  3.80286900e-03 -1.11151412e-02  1.12311756e-02  1.40622421e-03
 -3.82081210e-03  1.39175523e-02 -1.86602920e-02 -3.24355229e-03
  1.49886182e-03  3.34160496e-03 -2.23836419e-03 -1.33535601e-02
  6.32027164e-03  3.91552113e-02 -3.84742469e-02 -1.30257756e-02
  1.98645219e-02  8.26662965e-03 -2.17539892e-02 -5.44917258e-03
  5.62586356e-03 -7.55675044e-03  2.20162328e-03 -1.43665951e-02
  1.06650498e-02  4.07896889e-03  3.55312508e-03 -1.25637185e-02
  6.63197599e-03 -1.89118292e-02  1.31224645e-02  2.96879793e-04
 -5.33318489e-05 -2.94414203e-04  9.07180714e-04 -5.32332528e-03
  1.40950142e-04 -8.41117278e-03  1.02306008e-02  1.05974567e-03
  1.71222575e-02 -1.00211008e-03  1.62672624e-03  8.87680857e-04
 -7

In [12]:
calculate_info('minimal', 'ml', False, model_file=model_dir + "sally_ensemble_minimal", mode="score")
calculate_info('minimal', 'ml', True, model_file=model_dir + "sally_ensemble_minimal_tight", mode="score", batch_size=batch_size)


21:58 madminer.ml          INFO    Found ensemble with 10 estimators and expectations None
22:00 madminer.fisherinfor INFO    Evaluating rate Fisher information
22:02 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 1 / 9
22:02 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 2 / 9
22:02 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 3 / 9
22:03 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 4 / 9
22:03 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 5 / 9
22:03 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 6 / 9
22:03 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 7 / 9
22:04 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 8 / 9
22:04 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 9 / 9
22:04 madminer.ml          IN

In [13]:
calculate_info('phi', 'ml', True, model_file=model_dir + "sally_ensemble_phi_tight", mode="score", batch_size=batch_size)


22:08 madminer.ml          INFO    Found ensemble with 10 estimators and expectations None
22:09 madminer.fisherinfor INFO    Evaluating rate Fisher information
22:11 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 6 / 57
22:11 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 12 / 57
22:11 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 18 / 57
22:11 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 24 / 57
22:12 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 30 / 57
22:12 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 36 / 57
22:12 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 42 / 57
22:12 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 48 / 57
22:12 madminer.fisherinfor INFO    Evaluating kinematic Fisher information on batch 54 / 57


## Histogram info

In [14]:
filenames = ['phi', 'met', 'ptl', 'pta', 'deltaphi_lv', 'deltaphi_la']
observables = ['phi', 'et_miss', 'pt_l1', 'pt_a1', 'deltaphi_lmet', 'deltaphi_la']
bins = 25

for filename, obs in zip(filenames, observables):
    calculate_info('histo_' + filename, 'histo', True, observable=obs, nbins=bins, histrange=None)
    

In [10]:
calculate_info(
    'histo_pta_phi',
    'histo2d',
    True,
    observable1='pt_a1',
    nbins1=5,
    histrange1=[300.,800.],
    observable2='phi',
    nbins2=25,
    histrange2=None,
)


11:24 madminer.fisherinfor DEBUG   Automatic dynamic binning for observable 2: bin boundaries [-2.91722501 -2.70626957 -2.48096351 -2.25366483 -2.00183123 -1.65561451
 -1.48756969 -1.17247768 -0.88928512 -0.64519701 -0.38552266 -0.11059011
  0.15760237  0.42465332  0.68512001  0.94419876  1.22374113  1.49607868
  1.64501608  1.9698059   2.23454952  2.45812069  2.68859586  2.9066698 ]
/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:1165: RuntimeWarning: divide by zero encountered in true_divide
  inv_sigma = sanitize_array(1.0 / sigma)  # Shape (n_events,)
/Users/johannbrehmer/work/projects/madminer/madminer/madminer/morphing.py:697: RuntimeWarning: invalid value encountered in true_divide
  a.append(0.5 * np.log(benchmark_weights[:, i_pos] / benchmark_weights[:, i_neg]))
/Users/johannbrehmer/work/projects/madminer/madminer/madminer/morphing.py:695: RuntimeWarning: invalid value encountered in true_divide
  a.append(np.log(benchmark_weights[:, i_pos] /